In [1]:
import pickle

wordnet = {}
with open('../../data/wordnet.pickle', 'rb') as f:
    wordnet = pickle.load(f)
    
len(wordnet)

5059

In [4]:
import pandas as pd

df = pd.read_csv('../../data/preprocessed_train.csv')
df = df[['data', 'label']]

df.head()

,data,label
0,지금 너 스스로를 죽여달라고 애원하는 것인가 ? 아닙니다 . 죄송합니다 . 죽을 거...,0
1,길동경찰서입니다 . 시 분 마트에 폭발물을 설치할거다 . 네 ? 똑바로 들어 한번만...,0
2,너 되게 귀여운거 알지 ? 나보다 작은 남자는 첨봤어 . 그만해 . 니들 놀리는거 ...,3
3,어이 거기 예 ? ? 너 말이야 너 . 이리 오라고 무슨 일 . 너 옷 좋아보인다 ...,1
4,저기요 혹시 날이 너무 뜨겁잖아요 ? 저희 회사에서 이 선크림 파는데 한 번 손등에...,1


In [5]:
import random


def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)

    return new_words


def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0

    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            return new_words

    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
    return new_words


def random_deletion(words, p):
    if len(words) == 1:
        return words

    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]

    return new_words

In [8]:
def EDA(sentence, _type='rs', alpha=0.1, num_aug=9):
    words = sentence.split(' ')
    words = [word for word in words if word != '']
    num_words = len(words)

    augmented_sentences = []
    num_new_per_technique = int(num_aug / 4) + 1
    
    if _type == 'rs':
        n_rs = max(1, int(alpha * num_words))
        
        for _ in range(num_new_per_technique):
            a_words = random_swap(words, n_rs)
            augmented_sentences.append(' '.join(a_words))
            
    else:
        for _ in range(num_new_per_technique):
            a_words = random_deletion(words, alpha)
            augmented_sentences.append(' '.join(a_words))
            
    augmented_sentences = [sentence for sentence in augmented_sentences]
    random.shuffle(augmented_sentences)
    
    if num_aug >= 1:
        augmented_sentences = augmented_sentences[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentences)
        augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

    augmented_sentences.append(sentence)

    return augmented_sentences


In [9]:
df['data'][0]

'지금 너 스스로를 죽여달라고 애원하는 것인가 ? 아닙니다 . 죄송합니다 . 죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해 ? 진짜 죽여버리고 싶게 . 정말 잘못했습니다 . 너가 선택해 . 너가 죽을래 네 가족을 죽여줄까 . 죄송합니다 . 정말 잘못했습니다 . 너에게는 선택권이 없어 . 선택 못한다면 너와 네 가족까지 모조리 죽여버릴거야 . 선택 못하겠습니다 . 한번만 도와주세요 . 그냥 다 죽여버려야겠군 . 이의 없지 ? 제발 도와주세요 .'

In [23]:
# EDA(df['data'][0], _type='rs', num_aug=1)

rs_df = df
rs_df['data'] = rs_df['data'].apply(lambda it: EDA(it, _type='rs', alpha=0.2, num_aug=1)[0])

rs_df.head()

,data,label
0,가족을 없지 지금 못한다면 애원하는 제발 ? 아닙니다 . . 싶게 죽을 ? 혼자 죽...,0
1,마시죠 죽는거야 시 얘기한다 업무방해죄에 폭발물을 설치할거다 장난전화 네 ? 폭발물...,0
2,. 되게 귀여운거 가져봐 ? 나보다 작은 남자는 너 . 저 ? 니들 놀리는거 보태준...,3
3,어이 거기 예 피시방 ? 너 말이야 내놔 돈 이리 무슨 너 일 . . 넌 좋아보인다...,1
4,안 바른거 발라 ? 뜨겁잖아요 ? 저희 ? 열심히 손등에 한 ? 번 여기 발라보실래...,1


In [24]:
rs_df['data'][0]

'가족을 없지 지금 못한다면 애원하는 제발 ? 아닙니다 . . 싶게 죽을 ? 혼자 죽지 스스로를 네 너가 해 ? 잘못했습니다 휘말리게 . 선택 정말 . 선택 너가 선택해 . 너 죽을래 사건에 우리까지 죽여줄까 . 너와 . 정말 진짜 선택권이 너에게는 . . . 없어 죽여달라고 도와주세요 . 가족까지 모조리 이의 네 . 못하겠습니다 . 죽여버릴거야 도와주세요 . 그냥 다 죽여버려야겠군 잘못했습니다 한번만 죽여버리고 거면 것인가 죄송합니다 죄송합니다'

In [25]:
rs_df.to_csv('../../data/train_rs.csv')

In [26]:
rd_df = df
rd_df['data'] = rd_df['data'].apply(lambda it: EDA(it, _type='rd', alpha=0.2, num_aug=1)[0])

rd_df.head()

,data,label
0,가족을 못한다면 애원하는 제발 ? 아닙니다 . . ? 혼자 죽지 스스로를 네 너가 ...,0
1,얘기한다 업무방해죄에 폭발물을 장난전화 네 ? 폭발물 들어 . 진정하세요 한번만 길...,0
2,. 되게 귀여운거 ? 작은 너 저 ? 니들 놀리는거 보태준거 작은데 지영아 너가 어...,3
3,어이 거기 예 피시방 ? 너 말이야 내놔 돈 이리 너 일 . . 넌 좋아보인다 옷 ...,1
4,안 바른거 발라 ? 뜨겁잖아요 ? 저희 ? 열심히 한 ? 번 여기 발라보실래요 ? ...,1


In [27]:
rd_df['data'][0]

'가족을 못한다면 애원하는 제발 ? 아닙니다 . . ? 혼자 죽지 스스로를 네 너가 해 ? 잘못했습니다 휘말리게 . 선택 . 선택 너가 . 너 죽을래 사건에 우리까지 죽여줄까 . 너와 . 정말 선택권이 너에게는 . . 없어 죽여달라고 도와주세요 . 가족까지 모조리 이의 네 . 못하겠습니다 . 죽여버릴거야 도와주세요 . 그냥 다 죽여버려야겠군 한번만 죽여버리고 죄송합니다'

In [28]:
rd_df.to_csv('../../data/train_rd.csv')